## Steps
1. Read csv file as pandas dataframe/Create cost matrix
2. Column and row reduction - For each row, find the minimum value and subtract it from all elements in that row. Next, do the same for each column.
3. Draw the minimum number of horizontal and vertical lines to cover all the zeros in the matrix.
4. Find the smallest uncovered element, which is 1.

  * Subtract 1 from all uncovered elements.

  * Add 1 to all elements at the intersection of lines (none in this case).

  * Keep all other elements the same.
5. Now, draw the minimum number of lines to cover all zeros in this new matrix.
6. Find an assignment of zeros such that no two assignments share a row or column.

    * Note: if there are more residency programs than there are unique rankings, then a lottery system must be established to assign lottery numbers to each person, which gives different cost to each applicant, which allows for the algorithm to choose. The lottery number will be the dummy number of the doctor. The lottery number can also be calculated via GPA, and other stats.


In [7]:
#dependencies
import requests
import io
from io import StringIO
from google.colab import files
import pandas as pd
import numpy as np

In [8]:
#Lillian
def import_from_github(github_url):
  #if a regular url
  if 'github.com' in github_url and 'raw.githubusercontent.com' not in github_url:
    github_url= github_url.replace('github.com', 'raw.githubusercontent.com')
    github_url= github_url.replace('/blob', '/')

  if not github_url.endswith('.csv'):
    print("URL does not point to a CSV file")

  df= pd.read_csv(github_url)
  return df

In [9]:
#Lillian
def import_from_local():
  print("Please select a CSV file to upload:")
  uploaded = files.upload()

  if not uploaded:
    print("Error: No file selected")
    return None

  filename = list(uploaded.keys())[0]


  if not filename.lower().endswith('.csv'):
    print("Error: File is not a CSV file")
    return None

  df = pd.read_csv(io.BytesIO(uploaded[filename]))
  df = pd.read_csv(filename)
  return df

In [10]:
#Lillian
def import_csv_files(num_files=3):
  print(f'CSV File Importer for Google Colab - Import {num_files} Files\n')
  print(f'Please Import Doctor-Residency Rankings, Number of Positions per Residency, and/or Doctor Statisitcs')
  print('=====================================================================================================')

  dataframes = []
  for i in range(num_files):
    print('\n')
    print(f'Importing File {i+1} of {num_files}...')
    print(f'File {i+1}:')
    choice= input('Enter your choice (1 or 2):').strip()
    print('1. Import from GitHub URL')
    print('2. Import from Local File')
    print('\n')

    if choice == '1':
      url= input('Enter GitHub URL:').strip()
      df= import_from_github(url)
      if df is not None:
        dataframes.append(df)
      else:
        print(f'Failed to import file {i+1}. Invalid GitHub URL.')
    elif choice == '2':
      df= import_from_local()
      if df is not None:
        dataframes.append(df)
      else:
        print(f'Failed to import file {i+1}.')
    else:
      print('Invalid choice. Please enter 1 or 2.')
      i-=1
  return dataframes

In [11]:
dataframes = import_csv_files(num_files=3)

CSV File Importer for Google Colab - Import 3 Files

Please Import Doctor-Residency Rankings, Number of Positions per Residency, and/or Doctor Statisitcs


Importing File 1 of 3...
File 1:
Enter your choice (1 or 2):2
1. Import from GitHub URL
2. Import from Local File


Please select a CSV file to upload:


Saving 100_doctors_10_residency_programs - Sheet1 (1).csv to 100_doctors_10_residency_programs - Sheet1 (1).csv


Importing File 2 of 3...
File 2:
Enter your choice (1 or 2):2
1. Import from GitHub URL
2. Import from Local File


Please select a CSV file to upload:


Saving 100_doctor_stats - Sheet1 (1).csv to 100_doctor_stats - Sheet1 (1).csv


Importing File 3 of 3...
File 3:
Enter your choice (1 or 2):2
1. Import from GitHub URL
2. Import from Local File


Please select a CSV file to upload:


Saving num_positions - Sheet1 (1).csv to num_positions - Sheet1 (1).csv


In [12]:
dataframes[0]

,Name,Mayo Clinic College of Medicine and Science,University of California San Francisco,Vanderbilt University Medical Center,McGaw Medical Center of Northwestern University,Pennsylvania Hospital of Penn Medicine,University of Texas Southwestern Medical Center,New York-Presbyterian/Columbia University Irving Medical Center,Massachusetts General Hospital,Duke University Hospital,Baylor College of Medicine
0,Dee Gibbs,3,2,5,7,9,1,10,8,4,6
1,Carolyn Arnold,10,1,2,3,4,5,6,7,8,9
2,Savannah Daugherty,8,5,10,4,2,3,7,6,1,9
3,Millie Serrano,2,9,4,1,10,8,7,6,3,5
4,Christina Smith,9,2,10,6,7,8,3,4,1,5
...,...,...,...,...,...,...,...,...,...,...,...
95,Evangelina Nolan,3,9,8,2,7,1,5,4,10,6
96,Ashley York,10,3,2,8,7,6,4,5,9,1
97,Yolanda House,2,5,6,9,7,4,8,1,10,3
98,Annette Solomon,9,6,8,2,1,7,3,4,5,10


In [13]:
dataframes[1]

,Name,USMLE,GPA,Clinical Experience,Letter of Recommendation,Interpersonal Qualities
0,Dee Gibbs,275,3.08,10,4,5
1,Carolyn Arnold,276,3.68,7,4,5
2,Savannah Daugherty,208,3.94,5,9,8
3,Millie Serrano,220,3.62,9,8,5
4,Christina Smith,291,3.05,7,9,7
...,...,...,...,...,...,...
95,Evangelina Nolan,201,3.89,5,9,4
96,Ashley York,295,3.05,10,9,4
97,Yolanda House,240,3.91,10,6,8
98,Annette Solomon,280,3.53,8,10,5


In [14]:
dataframes[2]

,Unnamed: 0,Mayo Clinic College of Medicine and Science,University of California San Francisco,Vanderbilt University Medical Center,McGaw Medical Center of Northwestern University,Pennsylvania Hospital of Penn Medicine,University of Texas Southwestern Medical Center,New York-Presbyterian/Columbia University Irving Medical Center,Massachusetts General Hospital,Duke University Hospital,Baylor College of Medicine
0,Positions,10,9,7,6,13,18,4,15,3,5


In [16]:
#Lillian
def lottery_numbers(df, stats_df=None):
  """
  Caluclate lottery numbers based on normalized scores across all columns.
  If no stats are provided, assign random lottery numbers.

  Imput:
  df (Dataframe) - Doctor-Residency Rankings dataframe
  stats_df (Dataframe, optional) - Doctor stats dataframe with numerical values to be normalized


  Return:
  Series: Lottery numbers where lower number indicate better ranking
  """
  if stats_df is None:
    random_num = np.random.permutation(df.shape[0])
    return pd.Series(random_num) + df.shape[1]
  else:
    #drop the first column because those are the names of the doctor
    stats_norm = stats_df.copy().iloc[:, 1:]

    for column in stats_norm.columns:
      col_min= stats_norm[column].min()
      col_max= stats_norm[column].max()

      # if all values in columns are the same
      if col_max== col_min:
        stats_norm[column] = 1.0/len(stats_norm.columns)
      else:
        #We assume that schools weight each of the qualities equally (want a well arounded canidate)
        #Ex. GPA is worth 1/4th of your score
        #We normalize the values in each column to be between 0 and 1 and divide by number of columns
        stats_norm[column] = ((stats_norm[column] - col_min)/ (col_max - col_min))/len(stats_norm.columns)

      stats_rank= stats_norm.sum(axis=1)
      #lottery number to assign doctors (lower the number, the more likely they are to get in dummy position)
      #if doctors have same stats, then the doctor who comes first will have will be ranked first
      stats_rank= stats_rank.rank(ascending=True, method='first')
      stats_rank= [num + (df.shape[1]-1) for num in stats_rank]
      lottery_num= pd.Series(stats_rank).astype(int)

    return lottery_num

In [19]:
#Lillian
def matrix_prep(df, residency_positions= [], lottery_num= []):
  """
  Prepare the matrix for the Hungarian algorithm by duplicating columns
  for multiple positions and adding dummy positions with lottery numbers.

  Input:
  df (DataFrame) - Original dataframe of program preferences
  residency_positions (list) - List of positions available for each program
  lottery_num (Series/list) - Lottery numbers for each doctor

  Return:
  DataFrame - Prepared matrix with duplicated columns and dummy positions.
  """

  #number of dummy positions
  dummy_positions = df.shape[0]- sum(residency_position)


  original_columns = list(df.columns)
  new_columns_order = []

  #duplicate the columns
  for program, positions in zip(df.columns[1:], residency_position):
    new_columns_order.append(program)
    for position in range(2, positions+1):
        duplicate_col_name= f'{program}_{position}'
        new_columns_order.append(duplicate_col_name)
        df[duplicate_col_name] = df[program]

  df = df[new_columns_order]

  #add dummy rows
  for i in range(1, dummy_positions+1):
    df[f'Dummy_{i}']= 0

  # assign the lottery number in the dummy position
  for column in range(1, dummy_positions+1):
    df.iloc[:, -column] = lottery_num

  return df


Hard-coded matrix manipulation


1. Column and row reduction - For each row, find the minimum value and subtract it from all elements in that row. Next, do the same for each column.

3. Build a bipartite graph where edges represent zeros and we try to traverse the graph efficiently. The goal is to find an augmenting path that checks if the doctor can be matched to a residency slot. If the slot is taken, it recursively attempts to find the next available slot.


In [21]:
#Alex
def hungarian_matrix_reduction(df):
  '''
  Performs row and column reduction to create matrix with at least one zero in each row and column.
  '''

  matrix = df.to_numpy()

  #row reduction
  row_min = np.min(matrix, axis=1).reshape(-1, 1)
  row_reduced_matrix = matrix - row_min

  #column reduction
  col_min = np.min(row_reduced_matrix, axis=0).reshape(1, -1)
  reduced_matrix = row_reduced_matrix - col_min

  return reduced_matrix

[[ 2  2  2 ... 33 33 33]
 [ 9  9  9 ... 16 16 16]
 [ 7  7  7 ... 30 30 30]
 ...
 [ 1  1  1 ... 63 63 63]
 [ 8  8  8 ... 73 73 73]
 [ 7  7  7 ... 97 97 97]]


In [22]:
#Alex
def minimum_lines(reduced_matrix):
  '''
  Finds the minimum number of lines to cover all zeros in the matrix.
  Uses bipartite matching to deteemine if the current matrix contains a complete assignment.
  If not, it determines how many lines are needed to cover all zeros.


  Imput:
  reduced_matrix

  Output: result: int that specificies minimum number of lines to cover all zeros
  matchR: list where indices represent residency and value is the doctor matched to it (-1 if unmatched)

  '''
  num_rows, num_cols = reduced_matrix.shape

  #builds bipartite graph where edges represent zeros
  graph = {} # Creates a set of lists, where each element of the list is the row, and the contents of the list are the columns in that row which have a zero.
  for i in range(num_rows):
    graph[i] = []
    for j in range(num_cols): # In each column, if there is a 0 for that row, append it to the list in the index of the graph set.
      if reduced_matrix[i, j] == 0:
        graph[i].append(j)

  #bipartite depth search
  def bpm(bpGraph, u, seen, matchR): # function defined within a function
    '''bpGraph = dict where each list in a dictionary represents residencies a doctor can match to,
    u = current doctor,
    seen = T/F depending on whether residency positions have been visited during search,
    matchR = list where indices represent residency and value is the doctor matched to it (-1 if unmatched)'''
    for v in bpGraph[u]: # for residency possible for the doctor to be matched with
        if not seen[v]: # if the residency has not been seen and checks for existence of an edge
            seen[v] = True # change that that residency has been seen
            if matchR[v] < 0 or bpm(bpGraph, matchR[v], seen, matchR): # if doctor is unmatched to residency or the currently matched doctor can be reassigned to a different residency
                # attempts to find an augmenting path from already-matched doctor in order to iteratively optimize assignment
                matchR[v] = u # doctor takes that slot in the residency
                return True # return that the Doctor has been matched
    return False # Return the doctor is not matched

  # matchR: for every column, we see which row is matched to it (-1 if unmatched)
  matchR = [-1] * num_cols
  result = 0

  for u in range(num_rows):
      seen = [False] * num_cols
      if bpm(graph, u, seen, matchR):
          result += 1

  return result, matchR

(71, [90, 75, 64, 57, 53, 52, 41, -1, -1, -1, 84, 70, 51, 1, -1, -1, -1, -1, -1, 89, 86, 85, 83, 76, 24, -1, 38, 32, 29, 27, 16, 3, 98, 73, 61, 59, 48, 47, 37, 26, 15, 6, -1, -1, -1, 99, 95, 87, 67, 63, 55, 54, 9, 5, 0, -1, -1, -1, -1, -1, -1, -1, -1, 45, 44, 30, 28, 81, 79, 68, 65, 50, 43, 42, 36, 34, 31, 19, 18, 17, 14, 12, 7, 4, 2, 25, 23, 22, 20, 11, 72, -1, -1, -1, -1, -1, -1, -1, -1, -1])


In [ ]:
df

,Mayo Clinic College of Medicine and Science,Mayo Clinic College of Medicine and Science_2,Mayo Clinic College of Medicine and Science_3,Mayo Clinic College of Medicine and Science_4,Mayo Clinic College of Medicine and Science_5,Mayo Clinic College of Medicine and Science_6,Mayo Clinic College of Medicine and Science_7,Mayo Clinic College of Medicine and Science_8,Mayo Clinic College of Medicine and Science_9,Mayo Clinic College of Medicine and Science_10,...,Dummy_69,Dummy_70,Dummy_71,Dummy_72,Dummy_73,Dummy_74,Dummy_75,Dummy_76,Dummy_77,Dummy_78
0,3,3,3,3,3,3,3,3,3,3,...,44,44,44,44,44,44,44,44,44,44
1,10,10,10,10,10,10,10,10,10,10,...,27,27,27,27,27,27,27,27,27,27
2,8,8,8,8,8,8,8,8,8,8,...,41,41,41,41,41,41,41,41,41,41
3,2,2,2,2,2,2,2,2,2,2,...,49,49,49,49,49,49,49,49,49,49
4,9,9,9,9,9,9,9,9,9,9,...,86,86,86,86,86,86,86,86,86,86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3,3,3,3,3,3,3,3,3,3,...,16,16,16,16,16,16,16,16,16,16
96,10,10,10,10,10,10,10,10,10,10,...,89,89,89,89,89,89,89,89,89,89
97,2,2,2,2,2,2,2,2,2,2,...,74,74,74,74,74,74,74,74,74,74
98,9,9,9,9,9,9,9,9,9,9,...,84,84,84,84,84,84,84,84,84,84


In [23]:
#Lillian
def cover_zeros_efficiently(reduced_matrix):
  """
  Finds the minimum vertex cover using Konig's theorem.
  Identifies which rows and columns to cover to eliminate all zeros with minimum number of lines.
  """

  num_rows, num_cols = reduced_matrix.shape

  _, matchR= minimum_lines(reduced_matrix)

  #finding unmatched rows
  unmatched_rows=set()
  row_matches= [-1]*num_rows

  #we build row_matches from the column matches
  for i, j in enumerate(matchR):
    if i != -1:
      row_matches[i] = j

  for i in range(num_rows):
    if row_matches[i] == -1:
      unmatched_rows.add(i)

  visited_rows=set()
  visited_cols=set()
  queue= list(unmatched_rows)

  while queue:
    #get ride of the first in queue
    # u is current row
    u= queue.pop(0)
    visited_rows.add(u)

    #find all zeros in this row
    for v in range(num_cols):
      if reduced_matrix[u, v]== 0 and v not in visited_cols:
        visited_cols.add(v)

        #find the row matched to the column
        matched_row=matchR[v]
        if matched_row != -1 and matched_row not in visited_rows:
          queue.append(matched_row)

    #implementation of Konig's theorem
    # the minimum vertex cover is equal to unmatched row + columns reachable from unmatched rows
    covered_rows= set(range(num_rows))- visited_rows
    covered_cols= visited_cols

  return covered_rows, covered_cols # returns tuple of sets ({},{}})

In [24]:
# Amos
# define a function that does the following
''' 1. go thru the matrix and find uncovered values: subtract the minimum (k) from these values
2. if a value exists at the intersection of a row and column, add the minimum value (k) to it
3. reset the system and run the algorithm again'''

def subtract_min(matrix, covered_rows, covered_cols):
  # find k
    matrix_after_row_removal = np.delete(matrix, list(covered_rows), axis=0)
    matrix_after_col_removal = np.delete(matrix_after_row_removal, list(covered_cols), axis=1)

    # Check if the matrix after removal is empty
    if matrix_after_col_removal.size == 0:
        print("Matrix after removing rows and columns is empty. No adjustment needed.")
        return matrix

    # Find the minimal uncovered value
    k = np.min(matrix_after_col_removal)

    # Adjust the original matrix
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if i in covered_rows and j in covered_cols:
                matrix[i, j] += k  # Add k to double-covered cells
            elif (i in covered_rows and j not in covered_cols) or (i not in covered_rows and j in covered_cols):
                continue  # Skip single-covered elements
            else:
                matrix[i, j] -= k  # Subtract k from uncovered cells

    return matrix

103
{1, 10, 13, 21, 24, 33, 35, 39, 40, 46, 49, 51, 66, 69, 70, 71, 72, 74, 76, 77, 78, 80, 82, 83, 84, 85, 86, 88, 89} {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89}


array([[ 2,  2,  2, ..., 32, 32, 32],
       [10, 10, 10, ..., 16, 16, 16],
       [ 7,  7,  7, ..., 29, 29, 29],
       ...,
       [ 1,  1,  1, ..., 62, 62, 62],
       [ 8,  8,  8, ..., 72, 72, 72],
       [ 7,  7,  7, ..., 96, 96, 96]])

In [25]:
#Amos
from scipy.optimize import linear_sum_assignment



def assign_jobs(matrix, residency_position):
    '''
    Inputs: matrix: square matrix of data
    residency_position: list of number of positions per residency

    outputs: assignments: list of tuples (doctor_index, residency_index)
    '''

    matrix = np.array(matrix, dtype=float)

    total_residency_positions = sum(residency_position)
    num_dummy_columns = matrix.shape[1] - total_residency_positions
    print(f"Number of dummy columns: {num_dummy_columns}")

    row_indices, col_indices = linear_sum_assignment(matrix)
    assignments = []
    for doctor_idx, residency_idx in zip(row_indices, col_indices):
        if residency_idx >= total_residency_positions:
            # Assign to dummy columns if the residency index corresponds to a dummy
            assignments.append((doctor_idx, -1))
        else:
            # Assign to actual residency programs
            assignments.append((doctor_idx, residency_idx))

    return assignments

Number of dummy columns: 10
Assignments (Doctor Index -> Residency Index):
Doctor 0 -> Slot 45
Doctor 1 -> Slot 10
Doctor 2 -> Slot 37
Doctor 3 -> Slot 9
Doctor 4 -> Slot 11
Doctor 5 -> Slot 46
Doctor 6 -> Slot 32
Doctor 7 -> Slot 82
Doctor 8 -> Unassigned
Doctor 9 -> Slot 47
Doctor 10 -> Slot 84
Doctor 11 -> Slot 12
Doctor 12 -> Unassigned
Doctor 13 -> Slot 28
Doctor 14 -> Unassigned
Doctor 15 -> Slot 33
Doctor 16 -> Unassigned
Doctor 17 -> Slot 69
Doctor 18 -> Slot 70
Doctor 19 -> Slot 71
Doctor 20 -> Slot 58
Doctor 21 -> Slot 48
Doctor 22 -> Slot 87
Doctor 23 -> Unassigned
Doctor 24 -> Slot 56
Doctor 25 -> Slot 89
Doctor 26 -> Slot 34
Doctor 27 -> Slot 29
Doctor 28 -> Slot 68
Doctor 29 -> Slot 30
Doctor 30 -> Slot 19
Doctor 31 -> Slot 72
Doctor 32 -> Slot 13
Doctor 33 -> Slot 35
Doctor 34 -> Slot 60
Doctor 35 -> Slot 51
Doctor 36 -> Slot 74
Doctor 37 -> Unassigned
Doctor 38 -> Slot 27
Doctor 39 -> Slot 67
Doctor 40 -> Slot 57
Doctor 41 -> Slot 0
Doctor 42 -> Unassigned
Doctor 43 -> 

In [26]:

def organize_assignments(assignments, residency_positions: list):
    ''' The point of this function is to translate slots back into residencies
    Inputs: assignments: list of tuples
    residency_positions: list of number of positions per residency
    '''
    # Mapping logic
    residency_map = []
    residency_names = [f'Residency {i+1}' for i in range(len(residency_positions))]
    # Create a mapping of each slot's numerical index to its corresponding residency name.
    # e.g., with [5, 10], the first 10 slots map to 'Residency 1', and the next 5 map to 'Residency 2'.
    for index, number_of_positions in enumerate(residency_positions):
        for _ in range(number_of_positions):
            residency_map.append(residency_names[index])

    # Print the assignments with residency names like in previous function:
    print("Assignments (Doctor Index -> Residency Name):")
    for doctor_idx, slot_idx in assignments:
        if slot_idx == -1:
            print(f"Doctor {doctor_idx} -> Unassigned")
        else:
            try:
                residency_name = residency_map[slot_idx]
                print(f"Doctor {doctor_idx} -> {residency_name}")
            except IndexError:
                print(f"Doctor {doctor_idx} -> Error: Invalid Slot {slot_idx}")

    return residency_map # Return the map for further use if needed

chart = organize_assignments(assignments, residency_positions=residency_position)

Assignments (Doctor Index -> Residency Name):
Doctor 0 -> Residency 6
Doctor 1 -> Residency 2
Doctor 2 -> Residency 5
Doctor 3 -> Residency 1
Doctor 4 -> Residency 2
Doctor 5 -> Residency 6
Doctor 6 -> Residency 5
Doctor 7 -> Residency 9
Doctor 8 -> Unassigned
Doctor 9 -> Residency 6
Doctor 10 -> Residency 9
Doctor 11 -> Residency 2
Doctor 12 -> Unassigned
Doctor 13 -> Residency 4
Doctor 14 -> Unassigned
Doctor 15 -> Residency 5
Doctor 16 -> Unassigned
Doctor 17 -> Residency 8
Doctor 18 -> Residency 8
Doctor 19 -> Residency 8
Doctor 20 -> Residency 6
Doctor 21 -> Residency 6
Doctor 22 -> Residency 10
Doctor 23 -> Unassigned
Doctor 24 -> Residency 6
Doctor 25 -> Residency 10
Doctor 26 -> Residency 5
Doctor 27 -> Residency 4
Doctor 28 -> Residency 8
Doctor 29 -> Residency 4
Doctor 30 -> Residency 3
Doctor 31 -> Residency 8
Doctor 32 -> Residency 2
Doctor 33 -> Residency 5
Doctor 34 -> Residency 6
Doctor 35 -> Residency 6
Doctor 36 -> Residency 8
Doctor 37 -> Unassigned
Doctor 38 -> Resid

In [42]:
def execution(rank_df, stats_df, residency_position_df):
  lottery_num = lottery_numbers(rank_df, stats_df)
  residency_position = residency_position.iloc[0, 1:].tolist()
  df = matrix_prep(rank_df, residency_position, lottery_num)
  reduced = hungarian_matrix_reduction(df)
  lines, _ = minimum_lines(reduced)
  rows, col = cover_zeros_efficiently(reduced)
  matrix = subtract_min(reduced, rows, col)
  assignments = assign_jobs(matrix, residency_position)
  chart = organize_assignments(assignments, residency_positions=residency_position)
  return chart